# Simple Analyzer for Leak Detector

In [1]:
import pandas as pd
import re
import os

## Convert CSV to Dataframe

In [2]:
csv_file = './2023-11-30_inner_collector_leak_raw_wo2.csv'

extracted_df = pd.read_csv(csv_file)
extracted_df

,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,post_data,url_leaks,post_leaks,referer_leaks,leaks
0,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/ServiceLogin?servi...,GET,origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
1,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/InteractiveLogin?c...,GET,origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
2,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/v3/signin/identifi...,GET,origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
3,https://www.youtube.com/watch?v=kq5ayctloom,https://www.youtube.com/watch?v=kq5ayctloom,https://accounts.google.com/ServiceLogin?servi...,GET,origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
4,https://www.youtube.com/watch?v=kq5ayctloom,https://www.youtube.com/watch?v=kq5ayctloom,https://accounts.google.com/InteractiveLogin?c...,GET,origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
...,...,...,...,...,...,...,...,...,...,...,...
57,https://www.youtube.com/feed/history,https://www.youtube.com/feed/history,https://accounts.google.com/InteractiveLogin?c...,GET,strict-origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
58,https://www.youtube.com/feed/history,https://www.youtube.com/feed/history,https://accounts.google.com/v3/signin/identifi...,GET,strict-origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
59,https://www.youtube.com/@msnbc,https://www.youtube.com/@msnbc,https://accounts.google.com/ServiceLogin?servi...,GET,strict-origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks
60,https://www.youtube.com/@msnbc,https://www.youtube.com/@msnbc,https://accounts.google.com/InteractiveLogin?c...,GET,strict-origin-when-cross-origin,https://www.youtube.com/,NaN,"[('www.youtube.com',), ('urlencode', 'www.yout...",[],[],url_leaks


In [10]:
def print_leak_to_csv(df):
    """
    Check dataframe if there are colomn with name: url_leaks, referer_leaks,
    and post_leaks. If there are, then print the dataframe to csv file.

    :param df: dataframe

    :return: csv file with specific name
    """
    os.makedirs("data_seg", exist_ok=True)
    extract_date = re.findall(r'20(.*?)_', csv_file)
    extract_date = ''.join(extract_date)  # Convert the list to a string

    # if 'url_leaks' in df and 'referer_leaks' in df and 'post_leaks' in df:
    #     return df.to_csv(extract_date + '_print_raw_leaks.csv')
    if 'url_leaks' in df:
        return df.to_csv("data_seg/"+ extract_date + '_print_url_leaks.csv')
    elif 'referer_leaks' in df:
        return df.to_csv("data_seg/"+ extract_date + '_print_referer_leaks.csv')
    elif 'post_leaks' in df:
        return df.to_csv("data_seg/"+ extract_date + '_print_post_leaks.csv')


In [4]:
def extract_leak(data, leak):
    """extract data with when 'leak' row is not empty

    :param dataframe data: input dataframe
    :param str leak: column name

    :return: dataframe with non-empty 'leak' column
    """
    return data[(data[leak] != "[]")
                & (data[leak] != 'None')
                & (data[leak] != None)]

In [5]:
def drop_non_leak(data, leak):
    """drop data that not related to the leak

    :param data: dataframe
    :param leak: column name

    :return: dataframe with non-empty 'leak' column
    """
    data_seed = extract_leak(data, leak)
    if leak == 'url_leaks':
        return data_seed.drop(['post_leaks', 'ref_data', 'referer_leaks', 'post_data'], axis=1)
    elif leak == 'referer_leaks':
        return data_seed.drop(['url_leaks', 'post_leaks', 'post_data'], axis=1)
    elif leak == 'post_leaks':
        return data_seed.drop(['url_leaks', 'referer_leaks', 'ref_data'], axis=1)


## Leaks on Referer

We still need to compare with the referrer policy to make it accurate

In [18]:
# create a new dataframe with non-empty 'referer_leaks' column
referer_leaks_df = drop_non_leak(extracted_df, 'referer_leaks')
referer_leaks_df

,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,referer_leaks,leaks


In [19]:
# print to csv

print_leak_to_csv(referer_leaks_df)

In [ ]:
referer_leaks_df.count()

In [ ]:
referer_leaks_df.drop_duplicates(subset=['final_url']).count()

In [ ]:
referer_leaks_df.final_url.value_counts()

In [ ]:
referer_leaks_df.req_url.value_counts()

In [ ]:
referer_leaks_df.req_method.value_counts() 

### Google

In [ ]:
# list of referer leaks to google
referer_leaks_df[referer_leaks_df['req_url'].str.contains('google')]

In [ ]:
# counting on referer leaks to google
referer_leaks_df[referer_leaks_df['req_url'].str.contains('google')].count()

### META / Facebook

In [ ]:
# list of referer leaks to Meta or Facebook
referer_leaks_df[referer_leaks_df['req_url'].str.contains('meta') | referer_leaks_df['req_url'].str.contains('facebook')]

In [ ]:
# Counting on referer leaks to Meta or Facebook
referer_leaks_df[referer_leaks_df['req_url'].str.contains('meta') | referer_leaks_df['req_url'].str.contains('facebook')].count()

### Tiktok

In [ ]:
# list of referer leaks to tiktok
referer_leaks_df[referer_leaks_df['req_url'].str.contains('tiktok')] 

In [ ]:
# Counting on referer leaks to tiktok
referer_leaks_df[referer_leaks_df['req_url'].str.contains('tiktok')].count()

## Leaks on URL

In [20]:
# create a new dataframe with non-empty 'referer_leaks' column
url_leaks_df = drop_non_leak(extracted_df, 'url_leaks')
url_leaks_df

,init_url,final_url,req_url,req_method,ref_pol_data,url_leaks,leaks
0,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/ServiceLogin?servi...,GET,origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
1,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/InteractiveLogin?c...,GET,origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
2,https://www.youtube.com/watch?v=w9wyajjybuc,https://www.youtube.com/watch?v=w9wyajjybuc,https://accounts.google.com/v3/signin/identifi...,GET,origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
3,https://www.youtube.com/watch?v=kq5ayctloom,https://www.youtube.com/watch?v=kq5ayctloom,https://accounts.google.com/ServiceLogin?servi...,GET,origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
4,https://www.youtube.com/watch?v=kq5ayctloom,https://www.youtube.com/watch?v=kq5ayctloom,https://accounts.google.com/InteractiveLogin?c...,GET,origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
...,...,...,...,...,...,...,...
57,https://www.youtube.com/feed/history,https://www.youtube.com/feed/history,https://accounts.google.com/InteractiveLogin?c...,GET,strict-origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
58,https://www.youtube.com/feed/history,https://www.youtube.com/feed/history,https://accounts.google.com/v3/signin/identifi...,GET,strict-origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
59,https://www.youtube.com/@msnbc,https://www.youtube.com/@msnbc,https://accounts.google.com/ServiceLogin?servi...,GET,strict-origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks
60,https://www.youtube.com/@msnbc,https://www.youtube.com/@msnbc,https://accounts.google.com/InteractiveLogin?c...,GET,strict-origin-when-cross-origin,"[('www.youtube.com',), ('urlencode', 'www.yout...",url_leaks


In [21]:
# and print to csv

print_leak_to_csv(url_leaks_df)

In [ ]:
url_leaks_df.count()

In [ ]:
url_leaks_df.req_method.value_counts()

In [ ]:
url_leaks_df.drop_duplicates(subset=['final_url'])

### GOOGLE

In [ ]:
# count of url leaks to google
url_leaks_df[url_leaks_df['req_url'].str.contains('google')].count()

In [ ]:
# list of url leaks to google
url_leaks_df[url_leaks_df['req_url'].str.contains('google')]

### Facebook or Meta

In [ ]:
# count of url leaks to Meta or Facebook
url_leaks_df[url_leaks_df['req_url'].str.contains('meta') | url_leaks_df['req_url'].str.contains('facebook')].count()

In [ ]:
# list of url leaks to Meta or Facebook
url_leaks_df[url_leaks_df['req_url'].str.contains('meta') | url_leaks_df['req_url'].str.contains('facebook')]

### Tiktok

In [ ]:
url_leaks_df[url_leaks_df['req_url'].str.contains('tiktok')] 

### Leaks on Post Data

In [22]:
# create a new dataframe with non-empty 'post_leaks' column
post_leaks_df = drop_non_leak(extracted_df, 'post_leaks')
post_leaks_df

,init_url,final_url,req_url,req_method,ref_pol_data,post_data,post_leaks,leaks


In [ ]:
# print to csv

post_leaks_df['post_data'] = [x[:2048] for x in post_leaks_df['post_data']] # truncate post_data to 2048 characters
print_leak_to_csv(post_leaks_df)

In [ ]:
post_leaks_df.count()

In [ ]:
post_leaks_df.req_method.value_counts()	

## Leaks to Google, facebook, and Meta

In [ ]:
# list of post data leaks to google
post_leaks_df[post_leaks_df['req_url'].str.contains('google')]

In [ ]:
# list of post data leaks to Meta or Facebook
post_leaks_df[post_leaks_df['req_url'].str.contains('meta') | post_leaks_df['req_url'].str.contains('facebook')]

In [ ]:
# list of post data leaks to tiktok
post_leaks_df[post_leaks_df['req_url'].str.contains('tiktok')] 